In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
# from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, fullwidth_path_widget, interactive_pipeline_files


Automatic pdb calling has been turned OFF


In [2]:

# Create some placeholder widgets of varying sizes
widget1 = widgets.Button(description="Widget 1", layout=widgets.Layout(width='150px', height='100px'))
widget2 = widgets.Button(description="Widget 2", layout=widgets.Layout(width='200px', height='150px'))
widget3 = widgets.Button(description="Widget 3", layout=widgets.Layout(width='100px', height='200px'))
widget4 = widgets.Button(description="Widget 4", layout=widgets.Layout(width='250px', height='150px'))
widget5 = widgets.Button(description="Widget 5", layout=widgets.Layout(width='100px', height='100px'))
widget6 = widgets.Button(description="Widget 6", layout=widgets.Layout(width='150px', height='200px'))

# Define a GridBox layout with a varying number of columns
grid = widgets.GridBox(
    [widget1, widget2, widget3, widget4, widget5, widget6],
    layout=widgets.Layout(grid_template_columns="repeat(3, 200px)", grid_gap="10px")
)

# Display the grid
display(grid)

# _pipeline_jupyter_widget = interactive_pipeline_widget(curr_active_pipeline=curr_active_pipeline)
# display(_pipeline_jupyter_widget)
# _pipeline_jupyter_widget

GridBox(children=(Button(description='Widget 1', layout=Layout(height='100px', width='150px'), style=ButtonSty…

In [ ]:
# <i class="fa-solid fa-clone"></i>
# form_item_layout = ipywidgets.Layout(
#     display='flex',
#     flex_flow='row',
#     justify_content='space-between'
# )

icons = ['clock-rotate-left', 'folder-tree', 'rev', 'file-import', 'file-export', 'upload', 'download', 'file-zipper', 'box-archive', 'message', 'road', 'file-arrow-up', 'file-arrow-down']
rendering_icons = ['vector-square', 'draw-polygon', 'object-group']

button = widgets.Button(
    description='Click me',
	layout=widgets.Layout(flex='0 1 auto', width='auto', margin='0px'),
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
secondary_button = widgets.Button(
    description='',
	layout=widgets.Layout(flex='0 1 auto', width='auto', margin='0px'), # means it won't grow beyond its content's width, can shrink if needed, and its basis is auto.
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='clone' # (FontAwesome names without the `fa-` prefix)
)
another_button = widgets.Button(description='', layout=widgets.Layout(flex='0 1 auto', width='auto', margin='0px'), disabled=False, button_style='info', tooltip='Click me', icon='clone')

# widgets.HBox([button, secondary_button])

items_auto = [button, secondary_button, another_button]
box_layout = widgets.Layout(display='flex', flex_flow='row', align_items='stretch', width='70%')
box_auto = widgets.Box(children=items_auto, layout=box_layout)
box_auto